<a href="https://colab.research.google.com/github/GodoyLucas/king_of_rebirth/blob/main/Notebooks/King_of_Rebirth_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de desempenho dos players em função do input no campeonado de Call of Duty Warzone King of Rebirth

# Importanto bibliotecas e dados

In [23]:
import pandas as pd
import numpy as np

In [24]:
final = pd.read_csv('https://raw.githubusercontent.com/GodoyLucas/king_of_rebirth/main/Dados/pro.csv', sep = ";")
final.head()

,player,input,partida,kills,colocacao
0,GARCIA,mouse,1,4.0,31.0
1,GARCIA,mouse,2,2.0,14.0
2,GARCIA,mouse,3,2.0,27.0
3,GARCIA,mouse,4,6.0,5.0
4,GARCIA,mouse,5,4.0,4.0


# Verificando condições gerais dos dados

In [25]:
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player     210 non-null    object 
 1   input      210 non-null    object 
 2   partida    210 non-null    int64  
 3   kills      207 non-null    float64
 4   colocacao  208 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 8.3+ KB


É possível verificar que, apesar de 4 registros nulos em 'kills' e 'colocacao', as 8 partidas dos 44 players estão presentes no conjunto de dados. 

Os dados nulos são então substituídos por "0" e o tipo dados dos campos são convertidos para inteiro.

In [26]:
final.fillna(value = {'kills': 0, 'colocacao': 0}, inplace = True)
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   player     210 non-null    object 
 1   input      210 non-null    object 
 2   partida    210 non-null    int64  
 3   kills      210 non-null    float64
 4   colocacao  210 non-null    float64
dtypes: float64(2), int64(1), object(2)
memory usage: 8.3+ KB


In [27]:
final = final.astype({'kills': int, 'colocacao': int})
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   player     210 non-null    object
 1   input      210 non-null    object
 2   partida    210 non-null    int64 
 3   kills      210 non-null    int64 
 4   colocacao  210 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 8.3+ KB


# Premissas de análise

- A final do campeonato foi realizada em 8 partidas;
- Das 8 partidas, apenas as 5 melhores foram usadas para a classificação geral do campeonato;
- O estudo contempla todas as partidas;
- O campeonato ocorreu no modo solo, ou seja, 1 pessoa por time, competição individual;
- 44 players participaram. Portanto, as colocações nas partidas vão de 1 a 44, não sendo possível empate;
- Nem todos os players participaram e/ou reportaram os resultados de todas as partidas. Houve caso do player não querer jogar, caso de ser desconectado da partida por motivo adverso e caso de não querer reportar o resultado de uma determinada partida. Os resultados faltantes decorrentes desses casos, resultados nulos ou resultados impossíveis dado o contexto serão estimados com base nos resultados do player nas demais partidas.
- Para que a lógica da planilha utilizada para consolidação dos resultados no dia do campeonato não quebrasse, resultados não reportados foram registrados com colocação 44. Este estudo procura identificar os registros legítimos para tratar os que assim só tenham sido reportados pelo uso da planilha.
  - Regristros com colocação 44 com zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação na partida;
  - Regristros com colocação 44 com mais de zero kills só são considerados legítimos caso não haja outro registro nesta mesma colocação com kills registradas na mesma partida;

Considerando a impossibilidade de empate em qualquer que seja a colocação, o ideal dos resultados seria não haver mais que um player por colocação em cada umas das partidas. Abaixo é possível observar tal ocorrencia, além da ocorrência da colocação "0":

In [28]:
registros_com_mais_de_um_player_por_colocacao = pd.DataFrame()

for partida in range(1, 9):
  for colocacao in range(0, 45):
    if final.query('partida == @partida  & colocacao == @colocacao').player.count() > 1:
      registros_com_mais_de_um_player_por_colocacao = pd.concat([registros_com_mais_de_um_player_por_colocacao, final.query('partida == @partida  & colocacao == @colocacao')])

registros_com_mais_de_um_player_por_colocacao.groupby(['partida', 'colocacao'])[['player']].count()

player
partida colocacao        
1       40              2
        44              9
2       44              4
3       28              2
        44              5
4       44             11
5       0               2
        6               2
        32              2
        44              6

# Tratamento dos dados

Descrição do roteiro de tratamento dado as regras do jogo e as condições dos dados:

- Identificar e separar registros da colocação "0"
- Identificar e separar registros com mais de um player por colocação por partida;
- Identificar e separar registros legítimos da colocação "44";
- Consolidar registros legítimos sem  necessidade de tratamento;
- Consolidar registros para tratamento;
- Tratar registros;
- Consolidar dados tratados;

## Identificando e separando registros com colocação "0":

In [29]:
registros_com_colocacao_0 = final.query('colocacao == 0')
registros_com_colocacao_0

,player,input,partida,kills,colocacao
19,ANXIETY,mouse,5,0,0
24,RIVOTRIL,mouse,5,0,0


In [30]:
registros_sem_colocacao_0 = final[final.index.isin(list(set(final.index) - set(registros_com_colocacao_0.index)))]
registros_sem_colocacao_0.sort_values('colocacao')

,player,input,partida,kills,colocacao
120,HARDTREVS,controle,1,10,1
97,MESTRE DO ESCUDO,mouse,3,7,1
119,iErickziN,controle,5,14,1
23,RIVOTRIL,mouse,4,8,1
66,VIEIRA,controle,2,16,1
...,...,...,...,...,...
73,XAVIER,mouse,4,0,44
72,XAVIER,mouse,3,0,44
130,DIEGOSSAURO,controle,1,0,44
100,ZYH,controle,1,0,44


## Identificando e separando registros com mais de um player por colocação por partida:

In [31]:
registros_com_mais_de_um_player_por_colocacao = pd.DataFrame()

for partida in range(1, 9):
  for colocacao in range(0, 45):
    if registros_sem_colocacao_0.query('partida == @partida  & colocacao == @colocacao').player.count() != 1:
      registros_com_mais_de_um_player_por_colocacao = pd.concat([registros_com_mais_de_um_player_por_colocacao, registros_sem_colocacao_0.query('partida == @partida  & colocacao == @colocacao')])

registros_com_mais_de_um_player_por_colocacao.sort_values('colocacao', ascending = False).head()

,player,input,partida,kills,colocacao
197,GEPEGE,controle,3,0,44
168,INEWWS,mouse,4,0,44
28,NICE PADRIN,controle,4,0,44
73,XAVIER,mouse,4,0,44
83,FLEMYS,controle,4,0,44


In [32]:
registros_sem_colocacao_0_e_apenas_um_player_por_colocacao = registros_sem_colocacao_0[registros_sem_colocacao_0.index.isin(list(set(final.index) - set(registros_com_mais_de_um_player_por_colocacao.index)))]
registros_sem_colocacao_0_e_apenas_um_player_por_colocacao.sort_values('colocacao')

,player,input,partida,kills,colocacao
119,iErickziN,controle,5,14,1
97,MESTRE DO ESCUDO,mouse,3,7,1
23,RIVOTRIL,mouse,4,8,1
120,HARDTREVS,controle,1,10,1
66,VIEIRA,controle,2,16,1
...,...,...,...,...,...
142,THENEVES,controle,3,1,37
54,SAVIOLI,controle,5,0,38
32,DANTE,controle,3,2,38
31,DANTE,controle,2,2,38


## Identificando e separando registros legítimos da colocação "44":

In [33]:
registros_legitimos_44_colocacao = pd.DataFrame()

for partida in range(1, 9):
  if registros_com_mais_de_um_player_por_colocacao.query('partida == @partida & colocacao == 44 & kills > 0').player.count() == 1:
    registros_legitimos_44_colocacao = pd.concat([registros_legitimos_44_colocacao, registros_com_mais_de_um_player_por_colocacao.query('partida == @partida & colocacao == 44 & kills > 0')])

registros_legitimos_44_colocacao

,player,input,partida,kills,colocacao
143,THENEVES,controle,4,2,44


## Consolidando registros legítimos sem  necessidade de tratamento;

In [34]:
registros_legitimos = pd.concat([registros_legitimos_44_colocacao, registros_sem_colocacao_0_e_apenas_um_player_por_colocacao])
registros_legitimos


,player,input,partida,kills,colocacao
143,THENEVES,controle,4,2,44
0,GARCIA,mouse,1,4,31
1,GARCIA,mouse,2,2,14
2,GARCIA,mouse,3,2,27
3,GARCIA,mouse,4,6,5
...,...,...,...,...,...
204,KALORINE,controle,5,1,27
205,NINEXT,mouse,1,4,28
206,NINEXT,mouse,2,7,3
207,NINEXT,mouse,3,5,8


## Consolidando registros a serem tratados

In [35]:
registros_a_serem_tratados = final[final.index.isin(list(set(final.index) - set(registros_legitimos.index)))]
registros_a_serem_tratados.sort_values(['partida', 'colocacao'])

,player,input,partida,kills,colocacao
20,RIVOTRIL,mouse,1,0,40
180,ALAURAPS,mouse,1,0,40
55,SAVYTZ,controle,1,0,44
70,XAVIER,mouse,1,0,44
100,ZYH,controle,1,0,44
110,PANTERA,mouse,1,0,44
130,DIEGOSSAURO,controle,1,0,44
145,DAVID,#N/D,1,0,44
150,DANIBOY,#N/D,1,0,44
160,KAIKPCB,#N/D,1,0,44


## Tratando dados

In [36]:
base_para_tratamento = registros_legitimos[registros_legitimos.player.isin(list(registros_a_serem_tratados.player.unique()))].groupby(by = 'player')[['kills', 'colocacao']].mean().sort_values(by = 'kills', ascending = False)
base_para_tratamento

,kills,colocacao
player,,
RIVOTRIL,8.333333,17.666667
GETWAY,7.750000,17.500000
NICE PADRIN,7.666667,15.333333
FOTTLE,6.000000,13.250000
NINEXT,5.250000,14.250000
SAVYTZ,4.000000,11.333333
ANXIETY,4.000000,22.500000
KAIKPCB,3.750000,18.750000
NENEXFPS,3.666667,23.666667


In [37]:
registros_tratados = pd.merge(registros_a_serem_tratados, base_para_tratamento, how = 'inner', on = 'player')
registros_tratados.drop(['kills_x', 'colocacao_x'], axis = 1, inplace = True)
registros_tratados.rename(columns={'kills_y': 'kills', 'colocacao_y': 'colocacao'}, inplace = True)
registros_tratados

,player,input,partida,kills,colocacao
0,ANXIETY,mouse,5,4.000000,22.500000
1,RIVOTRIL,mouse,1,8.333333,17.666667
2,RIVOTRIL,mouse,5,8.333333,17.666667
3,NICE PADRIN,controle,3,7.666667,15.333333
4,NICE PADRIN,controle,4,7.666667,15.333333
5,GETWAY,controle,5,7.750000,17.500000
6,SAVYTZ,controle,1,4.000000,11.333333
7,SAVYTZ,controle,5,4.000000,11.333333
8,FOTTLE,controle,5,6.000000,13.250000
9,FLEMYS,controle,4,3.250000,24.250000


## Consolidando registros para análise

In [38]:
final_tratado = pd.concat([registros_legitimos, registros_tratados])
final_tratado

,player,input,partida,kills,colocacao
143,THENEVES,controle,4,2.00,44.00
0,GARCIA,mouse,1,4.00,31.00
1,GARCIA,mouse,2,2.00,14.00
2,GARCIA,mouse,3,2.00,27.00
3,GARCIA,mouse,4,6.00,5.00
...,...,...,...,...,...
19,KAIKPCB,#N/D,1,3.75,18.75
20,INEWWS,mouse,4,3.00,9.25
21,ALAURAPS,mouse,1,1.25,21.50
22,KALORINE,controle,4,1.25,25.75


# Analise

In [39]:
final_tratado.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
#N/D,2.583333,21.694444
controle,4.644928,17.211594
mouse,3.065278,19.076389


In [40]:
registros_legitimos.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
#N/D,2.700000,21.400000
controle,4.653846,17.221154
mouse,2.865385,19.365385


In [41]:
final.groupby(by = 'input')[['kills', 'colocacao']].mean()

,kills,colocacao
input,,
#N/D,1.500000,31.300000
controle,3.944000,21.192000
mouse,2.353846,22.230769
